In [1]:
import numpy as np
import pandas as pd
from tqdm import trange

from utils import *

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn import preprocessing

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset

In [2]:
class LSTM(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_layer, n_classes):
        super(LSTM, self).__init__()
        self.n_layer = n_layer
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(in_dim, hidden_dim, n_layer, batch_first=True)
        self.classifier = nn.Linear(hidden_dim, n_classes)

    def forward(self, x):
        out, (h_n, c_n) = self.lstm(x)
        # 此时可以从out中获得最终输出的状态h
        # x = out[:, -1, :]
        x = h_n[-1, :, :]
        x = self.classifier(x)
        return x

In [3]:
postive_rate = 0.5

In [4]:
filename = './data/money_laundrying_dataset/gx_data_{}.npy'.format(postive_rate)
data = np.load(filename, allow_pickle=True)[()]

X = data['X']
y = data['y']#[:,np.newaxis]
X = preprocessing.scale(X.squeeze())
X = X[:,:,np.newaxis]
X_train,X_test,y_train,y_test = train_test_split(X,y)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(1125, 1000, 1) (1125,) (375, 1000, 1) (375,)


In [5]:
data = np.load('./embedding_data_{}.npy'.format(postive_rate), allow_pickle=True)[()]
# data.keys()
X_train = data['X_train'][:,:,np.newaxis]
X_test = data['X_test'][:,:,np.newaxis]
y_train = data['y_train']
y_test = data['y_test']
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(1125, 64, 1) (1125,) (375, 64, 1) (375,)


In [6]:
X_train = torch.from_numpy(X_train).to(torch.float32)
y_train = torch.from_numpy(y_train)
print(X_train.size(), y_train.size())

X_test = torch.from_numpy(X_test).to(torch.float32)
y_test = torch.from_numpy(y_test)
print(X_test.size(), y_test.size())

torch.Size([1125, 64, 1]) torch.Size([1125])
torch.Size([375, 64, 1]) torch.Size([375])


In [7]:
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(dataset = train_data,
                         batch_size = 16,
                         shuffle=True)
test_data = TensorDataset(X_test, y_test)
test_loader = DataLoader(dataset = test_data,
                         batch_size = 16,
                         shuffle=True)

In [11]:
n_classes = 2
in_dim = X_train.shape[1]
hidden_dim = 512
n_layer = 1
max_epoch = 100
torch.cuda.set_device(1)

In [12]:
model = LSTM(in_dim, hidden_dim, n_layer, n_classes)
model.to('cuda')

LSTM(
  (lstm): LSTM(64, 512, batch_first=True)
  (classifier): Linear(in_features=512, out_features=2, bias=True)
)

In [47]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)
best_acc, best_prec, best_f1 = 0,0,0
train_losses = []
for epoch in range(1, max_epoch+1):
    #train
    print('\nEpoch: %d' % epoch)
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    y_pred, y_true = [], []
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to('cuda'), targets.to('cuda')
        optimizer.zero_grad()
        outputs = model(inputs.reshape(inputs.shape[0], inputs.shape[2], inputs.shape[1]))
        outputs = outputs.to('cuda')
        loss = criterion(outputs, targets)
        train_losses.append(float(loss.cpu()))
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        y_true += np.array(targets.cpu()).tolist()
        y_pred += np.array(predicted.cpu()).tolist()
        
        acc = accuracy_score(np.array(y_true), np.array(y_pred))
        prec = precision_score(np.array(y_true), np.array(y_pred))
        f1 = f1_score(np.array(y_true), np.array(y_pred))
        print(batch_idx, len(train_loader), 'Loss:%.3f | Acc:%.3f  Prec:%.3f  F1:%.3f' \
              % (train_loss/(batch_idx+1), acc, prec, f1))
    
#             print(batch_idx, len(test_loader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
#                 % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))


Epoch: 1
0 71 Loss:0.683 | Acc:0.562  Prec:0.562  F1:0.720
1 71 Loss:7.535 | Acc:0.500  Prec:0.562  F1:0.529
2 71 Loss:5.867 | Acc:0.417  Prec:0.562  F1:0.391
3 71 Loss:7.720 | Acc:0.406  Prec:0.469  F1:0.441
4 71 Loss:9.700 | Acc:0.400  Prec:0.438  F1:0.467
5 71 Loss:9.937 | Acc:0.427  Prec:0.469  F1:0.522
6 71 Loss:9.004 | Acc:0.482  Prec:0.537  F1:0.597
7 71 Loss:8.417 | Acc:0.492  Prec:0.542  F1:0.615
8 71 Loss:7.603 | Acc:0.486  Prec:0.542  F1:0.584
9 71 Loss:7.268 | Acc:0.481  Prec:0.542  F1:0.556
10 71 Loss:6.947 | Acc:0.483  Prec:0.542  F1:0.533
11 71 Loss:6.503 | Acc:0.484  Prec:0.542  F1:0.512
12 71 Loss:6.172 | Acc:0.486  Prec:0.536  F1:0.529
13 71 Loss:5.952 | Acc:0.491  Prec:0.539  F1:0.548
14 71 Loss:5.781 | Acc:0.487  Prec:0.528  F1:0.553
15 71 Loss:5.466 | Acc:0.492  Prec:0.531  F1:0.567
16 71 Loss:5.251 | Acc:0.500  Prec:0.531  F1:0.556
17 71 Loss:5.135 | Acc:0.503  Prec:0.531  F1:0.543
18 71 Loss:5.049 | Acc:0.500  Prec:0.531  F1:0.528
19 71 Loss:4.834 | Acc:0.506  P

28 71 Loss:1.315 | Acc:0.483  Prec:0.509  F1:0.487
29 71 Loss:1.322 | Acc:0.483  Prec:0.509  F1:0.479
30 71 Loss:1.303 | Acc:0.480  Prec:0.500  F1:0.482
31 71 Loss:1.289 | Acc:0.484  Prec:0.508  F1:0.496
32 71 Loss:1.283 | Acc:0.481  Prec:0.500  F1:0.498
33 71 Loss:1.282 | Acc:0.480  Prec:0.500  F1:0.490
34 71 Loss:1.273 | Acc:0.482  Prec:0.500  F1:0.484
35 71 Loss:1.257 | Acc:0.483  Prec:0.500  F1:0.491
36 71 Loss:1.252 | Acc:0.483  Prec:0.500  F1:0.498
37 71 Loss:1.240 | Acc:0.479  Prec:0.491  F1:0.498
38 71 Loss:1.242 | Acc:0.481  Prec:0.491  F1:0.492
39 71 Loss:1.261 | Acc:0.478  Prec:0.491  F1:0.485
40 71 Loss:1.249 | Acc:0.479  Prec:0.491  F1:0.491
41 71 Loss:1.251 | Acc:0.481  Prec:0.494  F1:0.499
42 71 Loss:1.247 | Acc:0.483  Prec:0.497  F1:0.507
43 71 Loss:1.237 | Acc:0.483  Prec:0.497  F1:0.501
44 71 Loss:1.236 | Acc:0.483  Prec:0.497  F1:0.496
45 71 Loss:1.224 | Acc:0.486  Prec:0.497  F1:0.492
46 71 Loss:1.212 | Acc:0.489  Prec:0.503  F1:0.501
47 71 Loss:1.205 | Acc:0.491  P

57 71 Loss:1.361 | Acc:0.505  Prec:0.509  F1:0.507
58 71 Loss:1.350 | Acc:0.505  Prec:0.509  F1:0.503
59 71 Loss:1.355 | Acc:0.503  Prec:0.509  F1:0.497
60 71 Loss:1.344 | Acc:0.503  Prec:0.509  F1:0.493
61 71 Loss:1.338 | Acc:0.505  Prec:0.512  F1:0.501
62 71 Loss:1.332 | Acc:0.507  Prec:0.516  F1:0.507
63 71 Loss:1.322 | Acc:0.507  Prec:0.516  F1:0.511
64 71 Loss:1.319 | Acc:0.507  Prec:0.516  F1:0.507
65 71 Loss:1.311 | Acc:0.507  Prec:0.516  F1:0.503
66 71 Loss:1.309 | Acc:0.506  Prec:0.513  F1:0.506
67 71 Loss:1.299 | Acc:0.507  Prec:0.517  F1:0.512
68 71 Loss:1.291 | Acc:0.508  Prec:0.517  F1:0.509
69 71 Loss:1.283 | Acc:0.510  Prec:0.517  F1:0.506
70 71 Loss:1.277 | Acc:0.509  Prec:0.517  F1:0.504

Epoch: 6
0 71 Loss:1.614 | Acc:0.250  Prec:0.250  F1:0.400
1 71 Loss:1.178 | Acc:0.344  Prec:0.250  F1:0.276
2 71 Loss:1.081 | Acc:0.396  Prec:0.250  F1:0.216
3 71 Loss:0.973 | Acc:0.500  Prec:0.531  F1:0.515
4 71 Loss:1.098 | Acc:0.512  Prec:0.542  F1:0.571
5 71 Loss:1.210 | Acc:0.51

17 71 Loss:1.258 | Acc:0.479  Prec:0.492  F1:0.457
18 71 Loss:1.228 | Acc:0.484  Prec:0.492  F1:0.445
19 71 Loss:1.242 | Acc:0.481  Prec:0.486  F1:0.458
20 71 Loss:1.226 | Acc:0.482  Prec:0.487  F1:0.473
21 71 Loss:1.236 | Acc:0.474  Prec:0.487  F1:0.457
22 71 Loss:1.219 | Acc:0.470  Prec:0.487  F1:0.444
23 71 Loss:1.232 | Acc:0.469  Prec:0.483  F1:0.455
24 71 Loss:1.216 | Acc:0.475  Prec:0.495  F1:0.475
25 71 Loss:1.195 | Acc:0.481  Prec:0.495  F1:0.468
26 71 Loss:1.183 | Acc:0.488  Prec:0.495  F1:0.462
27 71 Loss:1.202 | Acc:0.484  Prec:0.495  F1:0.451
28 71 Loss:1.188 | Acc:0.489  Prec:0.505  F1:0.470
29 71 Loss:1.197 | Acc:0.494  Prec:0.513  F1:0.486
30 71 Loss:1.185 | Acc:0.502  Prec:0.529  F1:0.507
31 71 Loss:1.172 | Acc:0.504  Prec:0.531  F1:0.517
32 71 Loss:1.169 | Acc:0.506  Prec:0.531  F1:0.510
33 71 Loss:1.161 | Acc:0.511  Prec:0.531  F1:0.506
34 71 Loss:1.156 | Acc:0.507  Prec:0.531  F1:0.496
35 71 Loss:1.184 | Acc:0.502  Prec:0.518  F1:0.496
36 71 Loss:1.183 | Acc:0.502  P

45 71 Loss:0.938 | Acc:0.511  Prec:0.528  F1:0.508
46 71 Loss:0.941 | Acc:0.511  Prec:0.527  F1:0.513
47 71 Loss:0.952 | Acc:0.508  Prec:0.521  F1:0.514
48 71 Loss:0.957 | Acc:0.505  Prec:0.521  F1:0.508
49 71 Loss:0.947 | Acc:0.511  Prec:0.521  F1:0.506
50 71 Loss:0.951 | Acc:0.507  Prec:0.521  F1:0.499
51 71 Loss:0.958 | Acc:0.507  Prec:0.520  F1:0.504
52 71 Loss:0.970 | Acc:0.506  Prec:0.517  F1:0.506
53 71 Loss:0.965 | Acc:0.508  Prec:0.517  F1:0.503
54 71 Loss:0.991 | Acc:0.505  Prec:0.517  F1:0.497
55 71 Loss:0.987 | Acc:0.503  Prec:0.517  F1:0.491
56 71 Loss:0.998 | Acc:0.504  Prec:0.519  F1:0.498
57 71 Loss:1.013 | Acc:0.505  Prec:0.520  F1:0.504
58 71 Loss:1.014 | Acc:0.504  Prec:0.517  F1:0.506
59 71 Loss:1.041 | Acc:0.501  Prec:0.517  F1:0.501
60 71 Loss:1.069 | Acc:0.497  Prec:0.517  F1:0.494
61 71 Loss:1.076 | Acc:0.495  Prec:0.512  F1:0.495
62 71 Loss:1.096 | Acc:0.492  Prec:0.506  F1:0.495
63 71 Loss:1.089 | Acc:0.494  Prec:0.506  F1:0.492
64 71 Loss:1.088 | Acc:0.498  P

...

66 71 Loss:1.337 | Acc:0.479  Prec:0.485  F1:0.486
67 71 Loss:1.327 | Acc:0.481  Prec:0.487  F1:0.491
68 71 Loss:1.318 | Acc:0.479  Prec:0.487  F1:0.487
69 71 Loss:1.311 | Acc:0.479  Prec:0.486  F1:0.490
70 71 Loss:1.302 | Acc:0.479  Prec:0.486  F1:0.489

Epoch: 91
0 71 Loss:1.107 | Acc:0.375  Prec:0.000  F1:0.000
1 71 Loss:1.042 | Acc:0.375  Prec:0.375  F1:0.375
2 71 Loss:0.940 | Acc:0.396  Prec:0.406  F1:0.473
3 71 Loss:1.039 | Acc:0.359  Prec:0.406  F1:0.388
4 71 Loss:0.993 | Acc:0.400  Prec:0.458  F1:0.478
5 71 Loss:1.083 | Acc:0.385  Prec:0.422  F1:0.478
6 71 Loss:1.063 | Acc:0.411  Prec:0.422  F1:0.450
7 71 Loss:1.097 | Acc:0.430  Prec:0.422  F1:0.425
8 71 Loss:1.057 | Acc:0.444  Prec:0.422  F1:0.403
9 71 Loss:1.055 | Acc:0.456  Prec:0.450  F1:0.453
10 71 Loss:1.093 | Acc:0.449  Prec:0.438  F1:0.464
11 71 Loss:1.113 | Acc:0.443  Prec:0.438  F1:0.440
12 71 Loss:1.105 | Acc:0.447  Prec:0.438  F1:0.422
13 71 Loss:1.094 | Acc:0.446  Prec:0.438  F1:0.441
14 71 Loss:1.074 | Acc:0.454  

20 71 Loss:1.424 | Acc:0.485  Prec:0.494  F1:0.477
21 71 Loss:1.440 | Acc:0.489  Prec:0.500  F1:0.494
22 71 Loss:1.413 | Acc:0.484  Prec:0.490  F1:0.497
23 71 Loss:1.464 | Acc:0.482  Prec:0.490  F1:0.486
24 71 Loss:1.480 | Acc:0.487  Prec:0.490  F1:0.478
25 71 Loss:1.489 | Acc:0.481  Prec:0.490  F1:0.465
26 71 Loss:1.546 | Acc:0.477  Prec:0.481  F1:0.469
27 71 Loss:1.618 | Acc:0.475  Prec:0.478  F1:0.477
28 71 Loss:1.626 | Acc:0.474  Prec:0.475  F1:0.483
29 71 Loss:1.651 | Acc:0.471  Prec:0.475  F1:0.473
30 71 Loss:1.693 | Acc:0.470  Prec:0.475  F1:0.464
31 71 Loss:1.685 | Acc:0.467  Prec:0.475  F1:0.455
32 71 Loss:1.716 | Acc:0.466  Prec:0.473  F1:0.462
33 71 Loss:1.811 | Acc:0.460  Prec:0.460  F1:0.460
34 71 Loss:1.803 | Acc:0.457  Prec:0.455  F1:0.463
35 71 Loss:1.824 | Acc:0.460  Prec:0.455  F1:0.457
36 71 Loss:1.889 | Acc:0.463  Prec:0.455  F1:0.452
37 71 Loss:1.983 | Acc:0.462  Prec:0.455  F1:0.445
38 71 Loss:2.008 | Acc:0.462  Prec:0.455  F1:0.438
39 71 Loss:1.999 | Acc:0.463  P

45 71 Loss:1.510 | Acc:0.496  Prec:0.487  F1:0.502
46 71 Loss:1.512 | Acc:0.495  Prec:0.485  F1:0.505
47 71 Loss:1.495 | Acc:0.497  Prec:0.490  F1:0.514
48 71 Loss:1.484 | Acc:0.497  Prec:0.490  F1:0.509
49 71 Loss:1.467 | Acc:0.500  Prec:0.490  F1:0.505
50 71 Loss:1.453 | Acc:0.500  Prec:0.491  F1:0.510
51 71 Loss:1.439 | Acc:0.500  Prec:0.491  F1:0.514
52 71 Loss:1.426 | Acc:0.496  Prec:0.491  F1:0.507
53 71 Loss:1.410 | Acc:0.501  Prec:0.500  F1:0.518
54 71 Loss:1.407 | Acc:0.501  Prec:0.500  F1:0.522
55 71 Loss:1.395 | Acc:0.501  Prec:0.500  F1:0.526
56 71 Loss:1.396 | Acc:0.499  Prec:0.500  F1:0.520
57 71 Loss:1.384 | Acc:0.498  Prec:0.500  F1:0.516
58 71 Loss:1.380 | Acc:0.499  Prec:0.502  F1:0.521
59 71 Loss:1.373 | Acc:0.500  Prec:0.504  F1:0.526
60 71 Loss:1.364 | Acc:0.500  Prec:0.504  F1:0.522
61 71 Loss:1.361 | Acc:0.499  Prec:0.504  F1:0.517
62 71 Loss:1.352 | Acc:0.498  Prec:0.502  F1:0.519
63 71 Loss:1.342 | Acc:0.500  Prec:0.505  F1:0.525
64 71 Loss:1.331 | Acc:0.503  P

69 71 Loss:1.343 | Acc:0.512  Prec:0.517  F1:0.522
70 71 Loss:1.342 | Acc:0.511  Prec:0.517  F1:0.520

Epoch: 98
0 71 Loss:1.238 | Acc:0.562  Prec:0.562  F1:0.720
1 71 Loss:1.353 | Acc:0.562  Prec:0.562  F1:0.720
2 71 Loss:1.110 | Acc:0.604  Prec:0.604  F1:0.753
3 71 Loss:1.114 | Acc:0.547  Prec:0.604  F1:0.667
4 71 Loss:1.034 | Acc:0.512  Prec:0.547  F1:0.642
5 71 Loss:0.972 | Acc:0.531  Prec:0.562  F1:0.667
6 71 Loss:0.930 | Acc:0.545  Prec:0.573  F1:0.683
7 71 Loss:0.908 | Acc:0.539  Prec:0.562  F1:0.681
8 71 Loss:0.903 | Acc:0.521  Prec:0.562  F1:0.646
9 71 Loss:0.877 | Acc:0.537  Prec:0.578  F1:0.667
10 71 Loss:0.894 | Acc:0.540  Prec:0.576  F1:0.672
11 71 Loss:0.896 | Acc:0.531  Prec:0.562  F1:0.667
12 71 Loss:0.935 | Acc:0.524  Prec:0.562  F1:0.645
13 71 Loss:0.963 | Acc:0.509  Prec:0.562  F1:0.621
14 71 Loss:1.097 | Acc:0.479  Prec:0.517  F1:0.593
15 71 Loss:1.071 | Acc:0.484  Prec:0.521  F1:0.602
16 71 Loss:1.083 | Acc:0.485  Prec:0.521  F1:0.588
17 71 Loss:1.054 | Acc:0.500  

22 71 Loss:1.342 | Acc:0.519  Prec:0.510  F1:0.545
23 71 Loss:1.318 | Acc:0.518  Prec:0.509  F1:0.552
24 71 Loss:1.309 | Acc:0.520  Prec:0.512  F1:0.562
25 71 Loss:1.291 | Acc:0.517  Prec:0.508  F1:0.564
26 71 Loss:1.287 | Acc:0.519  Prec:0.508  F1:0.556
27 71 Loss:1.279 | Acc:0.522  Prec:0.508  F1:0.549
28 71 Loss:1.256 | Acc:0.530  Prec:0.508  F1:0.544
29 71 Loss:1.236 | Acc:0.535  Prec:0.518  F1:0.558
30 71 Loss:1.222 | Acc:0.538  Prec:0.524  F1:0.569
31 71 Loss:1.208 | Acc:0.539  Prec:0.526  F1:0.576
32 71 Loss:1.198 | Acc:0.536  Prec:0.526  F1:0.566
33 71 Loss:1.184 | Acc:0.535  Prec:0.526  F1:0.558
34 71 Loss:1.170 | Acc:0.537  Prec:0.531  F1:0.568
35 71 Loss:1.150 | Acc:0.545  Prec:0.545  F1:0.583
36 71 Loss:1.147 | Acc:0.546  Prec:0.545  F1:0.588
37 71 Loss:1.135 | Acc:0.541  Prec:0.545  F1:0.579
38 71 Loss:1.121 | Acc:0.546  Prec:0.545  F1:0.576
39 71 Loss:1.126 | Acc:0.542  Prec:0.545  F1:0.567
40 71 Loss:1.119 | Acc:0.541  Prec:0.543  F1:0.571
41 71 Loss:1.117 | Acc:0.540  P

In [48]:
# test
# print('epoch:{} testing'.format(epoch))
model.eval()
test_loss = 0
correct = 0
total = 0
y_pred, y_true = [], []

with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to('cuda'), targets.to('cuda')
        outputs = model(inputs.reshape(inputs.shape[0], inputs.shape[2], inputs.shape[1]))
        outputs = outputs.to('cuda')
        loss = criterion(outputs, targets)
        test_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        y_true += np.array(targets.cpu()).tolist()
        y_pred += np.array(predicted.cpu()).tolist()

    acc = accuracy_score(np.array(y_true), np.array(y_pred))
    prec = precision_score(np.array(y_true), np.array(y_pred))
    f1 = f1_score(np.array(y_true), np.array(y_pred))
#     print('Loss:%.3f | Acc:%.3f  Prec:%.3f  F1:%.3f' % (train_loss/(batch_idx+1), acc, prec, f1))
#     if prec > best_prec:
#         best_prec, best_acc, best_f1 = prec, acc, f1
print('===============================================')
print('Test result | Acc:%.3f  Prec:%.3f  F1:%.3f'% (acc, prec, f1))


Test result | Acc:0.520  Prec:0.000  F1:0.000
